# Assessing the attitudes towards the proposed new ACMA powers to combat misinformation and disinformation


## Generate prompt

This function takes the submission and submission author (from doc name) as a
parameter and returns the formatted prompt to be sent to the LLM


In [ ]:
def prompt_formatted(submission_string: str, submission_author: str) -> str:    
    # Read the first file and set a string variable
    with open('prompt.txt', 'r') as file:
        prompt = file.read()
        
    with open('prompt_issues.md', 'r') as file:
        issues = file.read()

    with open('prompt_guidance_note.md', 'r') as file:
        guidance_note = file.read()

    with open('prompt_fact_sheet.md', 'r') as file:
        fact_sheet = file.read()

    prompt = prompt.replace('|issues|', issues)
    prompt = prompt.replace('|guidance_note|', guidance_note)
    prompt = prompt.replace('|fact_sheet|', fact_sheet)

    prompt += "\n\n***************************************** SUBMISSION START *****************************************\n\n"

    prompt += f"Submission from: {submission_author}\n\n"
    
    prompt += submission_string

    prompt += "\n\n***************************************** SUBMISSION END *****************************************\n\n"

    return prompt

## Get AI response

This function calls the AI model to elicit a response


In [ ]:
from az_client import call_ai, get_vector
from db.docs import DocumentManager
from db.db_instance import DBClient
import json
import os
import shutil

client = DBClient()
db = DocumentManager()

with open('function.json', 'r') as f:
    function = json.load(f)

def extract_name_from_filename(filename):
    parts = filename.split('-')
    name_parts = parts[1:]
    name = ' '.join(name_parts).split('.')[0]
    return name

def add_to_json(data, filename):
    if not os.path.isfile('./data/processed_data.json'):
        with open('./data/processed_data.json', 'w') as f:
            json.dump([], f)    
    with open('./data/processed_data.json', 'r') as f:
        previous_data = json.load(f)
        previous_data.append({filename: data})    
    with open('./data/processed_data.json', 'w') as f:
        json.dump(previous_data, f)

def process_files_in_directory(directory, completed_directory):
    # Check if the 'completed' directory exists, if not, create it
    if not os.path.exists(completed_directory):
        os.makedirs(completed_directory)
    for filename in os.listdir(directory)[:300]:
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath) and filename.endswith('.md'):
            try:
                with open(filepath, 'r') as file:
                    submission = file.read()
                sub_author = extract_name_from_filename(filename)
                prompt = prompt_formatted(submission, sub_author)
                response = call_ai(prompt, function)
                response["author"] = sub_author
                response["file_name"] = filename.replace('.md', '')

                vector = get_vector(submission)

                db.new_doc(response, vector, True)
                add_to_json(response, filename.split('-')[0])
                # Move the processed file to the 'completed' directory
                completed_filepath = os.path.join(completed_directory, filename)
                shutil.move(filepath, completed_filepath)
            except Exception as e:
                print(f"Error processing file: {filename}")
                print(e)
                continue

# Example usage
directory = './data/files/converted'
completed_directory = './data/files/completed'
process_files_in_directory(directory, completed_directory)
